In [ ]:
!pip install tensorflow==2.10.0

In [ ]:
!pip3 install -U segmentation-models

In [ ]:
# Import PyDrive and associated libraries.
# This only needs to be done once per notebook.
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once per notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np

DDSM_dir = "/content/drive/MyDrive/Project_shared/Datasets/mammography/CBIS-DDSM"
data = pd.read_csv(f'{DDSM_dir}/training/metadata.csv')

IMAGES = np.load(f'{DDSM_dir}/training/images.npy', allow_pickle=True)
MASKS = np.load(f'{DDSM_dir}/training/masks.npy', allow_pickle=True)

print(len(data))
print(IMAGES.shape)
print(MASKS.shape)

3568
(3568, 256, 256, 1)
(3568, 256, 256, 1)


In [ ]:
from sklearn.model_selection import train_test_split

# Split dataset into train and test sets (80/20 split)
x_train, x_test, y_train, y_test = train_test_split(IMAGES, MASKS, test_size=0.2, random_state=42)

# Split training set into train and validation sets (80/20 split)
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

In [ ]:
from segmentation_models import Unet
import segmentation_models as s
from segmentation_models.losses import bce_jaccard_loss , DiceLoss
from segmentation_models.metrics import iou_score, IOUScore

#s.Unet()
#N = x_train.shape[-1]
'''
model = Unet('resnet101',
             input_shape=(256, 256, 1),
             classes = 1 ,
             activation='sigmoid',
             encoder_weights=None,
             #encoder_freeze=False, 
             encoder_features='default',
             #decoder_block_type='upsampling', 
             decoder_filters=(256, 128, 64, 32,16))

model.compile('Adam', loss=bce_jaccard_loss, metrics=[iou_score])
'''

Segmentation Models: using `keras` framework.


"\nmodel = Unet('resnet101',\n             input_shape=(256, 256, 1),\n             classes = 1 ,\n             activation='sigmoid',\n             encoder_weights=None,\n             #encoder_freeze=False, \n             encoder_features='default',\n             #decoder_block_type='upsampling', \n             decoder_filters=(256, 128, 64, 32,16))\n\nmodel.compile('Adam', loss=bce_jaccard_loss, metrics=[iou_score])\n"

In [ ]:
import keras
from keras.models import Model
from keras.layers import Conv2D, MaxPooling2D, Input, Conv2DTranspose, Concatenate, BatchNormalization, UpSampling2D, concatenate
from keras.layers import  Dropout, Activation
from keras.optimizers import Adam, SGD
from keras.layers import LeakyReLU
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from keras import backend as K
from keras.utils import plot_model
import tensorflow as tf
import glob
import random
import cv2
from random import shuffle
from tensorflow.keras.metrics import BinaryAccuracy, Precision, MeanIoU, Recall, AUC
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:

def unet(input_size = (256,256,1)):
    inputs = Input(input_size)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(inputs)
    conv1 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)

    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool1)
    conv2 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)

    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool2)
    conv3 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool3)
    conv4 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling2D(pool_size=(2, 2))(drop4)

    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(pool4)
    conv5 = Conv2D(1024, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv5)
    drop5 = Dropout(0.5)(conv5)

    up6 = Conv2D(512, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(drop5))
    merge6 = concatenate([drop4,up6], axis = 3)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge6)
    conv6 = Conv2D(512, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv6)

    up7 = Conv2D(256, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv6))
    merge7 = concatenate([conv3,up7], axis = 3)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge7)
    conv7 = Conv2D(256, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv7)

    up8 = Conv2D(128, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv7))
    merge8 = concatenate([conv2,up8], axis = 3)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge8)
    conv8 = Conv2D(128, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv8)

    up9 = Conv2D(64, 2, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(UpSampling2D(size = (2,2))(conv8))
    merge9 = concatenate([conv1,up9], axis = 3)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(merge9)
    conv9 = Conv2D(64, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
    #conv9 = Conv2D(2, 3, activation = 'relu', padding = 'same', kernel_initializer = 'he_normal')(conv9)
   
    conv10 = Conv2D(1, 1, activation = 'sigmoid')(conv9)

    model = Model(inputs = [inputs], outputs = [conv10])

    return model

In [ ]:
model1 = unet(input_size = (256,256,1))

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

mat = IOUScore(smooth=100)
model1.compile('Adam', loss=bce_jaccard_loss, metrics=["binary_accuracy", mat])
callback= [ModelCheckpoint('/content/drive/MyDrive/models/m1.h5',verbose =1, save_best_only=True),
    EarlyStopping(patience = 5, verbose =1, restore_best_weights=True ,monitor = "val_loss")]

In [ ]:
model1.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 256, 256, 1  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 256, 256, 64  640         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 conv2d_1 (Conv2D)              (None, 256, 256, 64  36928       ['conv2d[0][0]']                 
                                )                                                             

In [ ]:
y_train = y_train.astype('float32')
y_val = y_val.astype('float32')

train_steps = x_train.shape[0] 
test_steps = x_val.shape[0] 

model1.fit(x=x_train, 
          y=y_train, 
          epochs=100,
          steps_per_epoch = train_steps, 
          validation_steps= test_steps ,
          callbacks = callback,
          batch_size = 1, 
          validation_data=(x_val, y_val))

Epoch 1/100
2280/2283 [============================>.] - ETA: 0s - loss: 1.1542 - binary_accuracy: 0.9889 - iou_score: 0.0934
Epoch 1: val_loss improved from inf to 1.04475, saving model to /content/drive/MyDrive/models/m1.h5
2283/2283 [==============================] - 56s 20ms/step - loss: 1.1541 - binary_accuracy: 0.9889 - iou_score: 0.0934 - val_loss: 1.0448 - val_binary_accuracy: 0.9880 - val_iou_score: 0.0963
Epoch 2/100
2280/2283 [============================>.] - ETA: 0s - loss: 1.0375 - binary_accuracy: 0.9901 - iou_score: 0.1001
Epoch 2: val_loss did not improve from 1.04475
2283/2283 [==============================] - 39s 17ms/step - loss: 1.0375 - binary_accuracy: 0.9901 - iou_score: 0.1000 - val_loss: 1.0602 - val_binary_accuracy: 0.9880 - val_iou_score: 0.0476
Epoch 3/100
2281/2283 [============================>.] - ETA: 0s - loss: 1.0618 - binary_accuracy: 0.9901 - iou_score: 0.0795
Epoch 3: val_loss did not improve from 1.04475
2283/2283 [==============================]

In [ ]:
from tensorflow.keras.models import load_model
model1.load_model()

In [ ]:
predicted = model1.predict(x_test)

In [ ]:
from matplotlib import pyplot as plt

def plot_image(img):
  plt.imshow(img.reshape(256,256), interpolation='nearest',cmap='gray')
  plt.show()
  return plt

In [ ]:
plot_image(predicted[75])
plot_image(x_test[75])
plot_image(y_test[75])